In [1]:
import pandas as pd
import numpy as np


In [2]:
schools = pd.read_csv("Resource/schools_complete.csv")
schools = pd.DataFrame(data= schools)
schools.head()


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
students = pd.read_csv("Resource/students_complete.csv")
students = pd.DataFrame(data= students)
students.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## District Summary

In [4]:
district = {}
district = pd.DataFrame()

district['Total Schools']= [schools['School ID'].nunique()]
district['Total Students']= [students['Student ID'].nunique()]
district['Total Budget']= [schools['budget'].sum()]
district['Average Math Score']= [students['math_score'].mean()]
district['Average Reading Score']= [students['reading_score'].mean()]
district['% Passing Math']=students.loc[students['math_score']>=70]['math_score'].count()/students['math_score'].count()*100
district['% Passing Reading']=students.loc[students['reading_score']>=70]['reading_score'].count()/students['reading_score'].count()*100
district['Overall Passing Rate']= (district['% Passing Math']+district['% Passing Reading'])/2




district

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [5]:
district.style.format({"Total Students": "{:,.0f}",
                        "Total Budget": "${:,.2f}",
                      "%Passing Math": "{:.1%}",
                       "%Passing Reading": "{:.1%}"})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.9854,81.8778,74.9809,85.8055,80.3932


## School Summary

In [7]:
students['pass reading']= np.where(students['reading_score']>=70,1,0)
students['pass math']= np.where(students['math_score']>=70,1,0)
students.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,pass reading,pass math
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,1
1,1,Victor Smith,M,12th,Huang High School,94,61,1,0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,1,0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,1,1


In [8]:
stu_agg = students.groupby(['school_name']).agg({'Student ID':'count',
                                       'reading_score':'mean',
                                       'math_score':'mean',
                                      'pass math':'sum',
                                      'pass reading':'sum'})

In [36]:
stu_agg['% Passing Math']= stu_agg['pass math']/stu_agg['Student ID']
stu_agg['% Passing reading']= stu_agg['pass reading']/stu_agg['Student ID']
stu_agg['Overall Passing Rate']= (stu_agg['% Passing Math']+stu_agg['% Passing reading'])/2
#a.style.format( "{:.1%}")
stu_agg.head()

,Student ID,reading_score,math_score,pass math,pass reading,% Passing Math,% Passing reading,Overall Passing Rate
school_name,,,,,,,,
Bailey High School,4976,81.033963,77.048432,3318,4077,0.666801,0.819333,0.743067
Cabrera High School,1858,83.975780,83.061895,1749,1803,0.941335,0.970398,0.955867
Figueroa High School,2949,81.158020,76.711767,1946,2381,0.659885,0.807392,0.733639
Ford High School,2739,80.746258,77.102592,1871,2172,0.683096,0.792990,0.738043
Griffin High School,1468,83.816757,83.351499,1371,1426,0.933924,0.971390,0.952657


In [37]:
school_summary = schools.set_index('school_name').join(stu_agg)
school_summary.reset_index(inplace=True)
#school_summary.head()

In [38]:
school_summary.rename(columns = {'Student ID':'total student','budget':'total school budget','type':'school type'},inplace = True)
school_summary['per student budget']= school_summary['total school budget']/school_summary['total student']
school_summary.head()

,school_name,School ID,school type,size,total school budget,total student,reading_score,math_score,pass math,pass reading,% Passing Math,% Passing reading,Overall Passing Rate,per student budget
0,Huang High School,0,District,2917,1910635,2917,81.182722,76.629414,1916,2372,0.656839,0.813164,0.735002,655.0
1,Figueroa High School,1,District,2949,1884411,2949,81.158020,76.711767,1946,2381,0.659885,0.807392,0.733639,639.0
2,Shelton High School,2,Charter,1761,1056600,1761,83.725724,83.359455,1653,1688,0.938671,0.958546,0.948609,600.0
3,Hernandez High School,3,District,4635,3022020,4635,80.934412,77.289752,3094,3748,0.667530,0.808630,0.738080,652.0
4,Griffin High School,4,Charter,1468,917500,1468,83.816757,83.351499,1371,1426,0.933924,0.971390,0.952657,625.0


In [41]:
school_summary_cut = school_summary[['school_name','school type','total student','total school budget',
                                     'per student budget','math_score'
                                     ,'reading_score','% Passing Math','% Passing reading','Overall Passing Rate']]
#formatting
school_summary_cut.style.format({'total student': '{:,}', 
                          "total school budget": "${:,}",
                          "per student budget":'{:,}',
                           "math_score":'{:,.2f}' ,
                        "reading_score":'{:,.2f}',
                         "% Passing Math":'{:.1%}' ,
                                 "% Passing reading":'{:.1%}',
                                 "Overall Passing Rate":'{:.1%}'
                                })



,school_name,school type,total student,total school budget,per student budget,math_score,reading_score,% Passing Math,% Passing reading,Overall Passing Rate
0,Huang High School,District,"2,917","$1,910,635",655.0,76.63,81.18,65.7%,81.3%,73.5%
1,Figueroa High School,District,"2,949","$1,884,411",639.0,76.71,81.16,66.0%,80.7%,73.4%
2,Shelton High School,Charter,"1,761","$1,056,600",600.0,83.36,83.73,93.9%,95.9%,94.9%
3,Hernandez High School,District,"4,635","$3,022,020",652.0,77.29,80.93,66.8%,80.9%,73.8%
4,Griffin High School,Charter,"1,468","$917,500",625.0,83.35,83.82,93.4%,97.1%,95.3%
5,Wilson High School,Charter,"2,283","$1,319,574",578.0,83.27,83.99,93.9%,96.5%,95.2%
6,Cabrera High School,Charter,"1,858","$1,081,356",582.0,83.06,83.98,94.1%,97.0%,95.6%
7,Bailey High School,District,"4,976","$3,124,928",628.0,77.05,81.03,66.7%,81.9%,74.3%
8,Holden High School,Charter,427,"$248,087",581.0,83.80,83.81,92.5%,96.3%,94.4%
9,Pena High School,Charter,962,"$585,858",609.0,83.84,84.04,94.6%,95.9%,95.3%


## Top Performing Schools (By Passing Rate)

In [42]:
top5= school_summary_cut.sort_values(by=['Overall Passing Rate'],ascending = False).head(5)
top5

,school_name,school type,total student,total school budget,per student budget,math_score,reading_score,% Passing Math,% Passing reading,Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.952905
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.952703
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.952037



## Bottom Performing Schools (By Passing Rate)

In [43]:
bottom5= school_summary_cut.sort_values(by=['Overall Passing Rate'],ascending = True).head(5)
bottom5

,school_name,school type,total student,total school budget,per student budget,math_score,reading_score,% Passing Math,% Passing reading,Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.732933
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.735002
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.736400
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043


## Math Scores by Grade

In [58]:
#Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school
math_by_grade = students.groupby(['school_name','grade']).agg({'math_score':'mean'})
math_by_grade= math_by_grade.unstack()

math_by_grade.head(10)
#

math_score                                 
grade                       10th       11th       12th        9th
school_name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455

## Reading Scores by Grade

In [118]:
nine = students.loc[students['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
ten = students.loc[students['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
eleven = students.loc[students['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
twelve = students.loc[students['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

reading_by_grade = pd.DataFrame({'9th':nine,'10th':ten,'11th':eleven,'12th':twelve})
reading_by_grade.reset_index()

#reading_by_grade.style.format({'9th': '{:.1f}', 
#                              "10th": '{:.1f}', 
#                              "11th": "{:.1f}", 
#                              "12th": "{:.1f}"})

reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [120]:
print(school_summary_cut['per student budget'].min(),school_summary_cut['per student budget'].max())

578.0 655.0


In [151]:
a=round(((school_summary_cut['per student budget'].max()-school_summary_cut['per student budget'].min())/4
         
     ),0)
a

19.0

In [153]:
bin = (577,577+a,577+a*2,577+a*3,577+a*5)
bin

(577, 596.0, 615.0, 634.0, 672.0)

In [137]:
school_student = schools.join(students.set_index('school_name'), on = 'school_name')
school_student.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score,pass reading,pass math
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,0,1
0,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,1,0
0,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,1,0
0,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,0,0
0,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,1,1


In [168]:
bin = [0, 585, 615, 645, 675]
bins =pd.cut(school_summary_cut['per student budget'],bin)
score_by_spending = school_summary_cut.groupby(bins).agg({'math_score':'mean','reading_score':'mean','% Passing Math':'mean',
                                     '% Passing reading':'mean'})
score_by_spending['Overall Passing Rate']=(score_by_spending['% Passing Math']+score_by_spending['% Passing reading']
                                          )/2
score_by_spending.style.format({'math_score':'{:.2f}', 'reading_score':'{:.2f}',
                                '% Passing Math':'{:.2%}',
                                '% Passing reading':'{:.2%}',
                                'Overall Passing Rate':'{:.2%}'})

#reading_by_grade.style.format({'9th': '{:.1f}', 
#                              "10th": '{:.1f}', 
#                              "11th": "{:.1f}", 
#                              "12th": "{:.1f}"})

,math_score,reading_score,% Passing Math,% Passing reading,Overall Passing Rate
per student budget,,,,,
"(0, 585]",83.46,83.93,93.46%,96.61%,95.04%
"(585, 615]",83.60,83.89,94.23%,95.90%,95.07%
"(615, 645]",79.08,81.89,75.67%,86.11%,80.89%
"(645, 675]",77.00,81.03,66.16%,81.13%,73.65%


## Scores by School Size

In [170]:
print(school_summary_cut['total student'].min(),school_summary_cut['total student'].max())

427 4976


In [173]:
bin2 = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
bins2 =pd.cut(school_summary_cut['total student'],bin2,labels = group_names)
score_by_size = school_summary_cut.groupby(bins2).agg({'math_score':'mean','reading_score':'mean','% Passing Math':'mean',
                                     '% Passing reading':'mean'})
score_by_size['Overall Passing Rate']=(score_by_size['% Passing Math']+score_by_size['% Passing reading']
                                          )/2
score_by_size.style.format({'math_score':'{:.2f}', 'reading_score':'{:.2f}',
                                '% Passing Math':'{:.2%}',
                                '% Passing reading':'{:.2%}',
                                'Overall Passing Rate':'{:.2%}'})


,math_score,reading_score,% Passing Math,% Passing reading,Overall Passing Rate
total student,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,95.20%
Large (2000-5000),77.75,81.34,69.96%,82.77%,76.36%


## Scores by School Type

In [177]:
score_by_type = school_summary_cut.groupby(['school type']).agg({'math_score':'mean','reading_score':'mean','% Passing Math':'mean',
                                     '% Passing reading':'mean'})
score_by_type['Overall Passing Rate']=(score_by_type['% Passing Math']+score_by_type['% Passing reading']
                                          )/2
score_by_type.style.format({'math_score':'{:.1f}', 'reading_score':'{:.1f}',
                                '% Passing Math':'{:.2%}',
                                '% Passing reading':'{:.2%}',
                                'Overall Passing Rate':'{:.2%}'})

,math_score,reading_score,% Passing Math,% Passing reading,Overall Passing Rate
school type,,,,,
Charter,83.5,83.9,93.62%,96.59%,95.10%
District,77.0,81.0,66.55%,80.80%,73.67%
